In [1]:
from brightway2 import *
import bw2data as bd
import pandas as pd
import numpy as np
import os

# BW setup

In [3]:
#Set Project
projects.set_current('plca_metals')

In [4]:
list(databases)

['biosphere3',
 'ecoinvent-3.10-cutoff',
 'NZE_2022',
 'NZE_2025',
 'NZE_2030',
 'NZE_2035',
 'NZE_2040',
 'NZE_2045',
 'NZE_2050',
 'biosphere3_spatialized_flows',
 'NZE_2050 regionalized',
 'SPS_2022',
 'SPS_2025',
 'SPS_2030',
 'SPS_2035',
 'SPS_2040',
 'SPS_2045',
 'SPS_2050',
 'SPS_2022 regionalized',
 'SPS_2025 regionalized',
 'SPS_2030 regionalized',
 'SPS_2035 regionalized',
 'SPS_2040 regionalized',
 'SPS_2045 regionalized',
 'SPS_2050 regionalized',
 'NZE_2022 regionalized',
 'NZE_2025 regionalized',
 'NZE_2030 regionalized',
 'NZE_2035 regionalized',
 'NZE_2040 regionalized',
 'NZE_2045 regionalized']

In [5]:
# Import IW+2.1. methods 
IW_METHODS = [method for method in bd.methods if "impact world+" in " ".join(method).lower()]
IW_METHODS

[('IMPACT World+ v2.0.1, footprint version',
  'climate change',
  'carbon footprint'),
 ('IMPACT World+ v2.0.1, footprint version',
  'ecosystem quality',
  'remaining ecosystem quality damage'),
 ('IMPACT World+ v2.0.1, footprint version',
  'energy resources: non-renewable',
  'fossil and nuclear energy use'),
 ('IMPACT World+ v2.0.1, footprint version',
  'human health',
  'remaining human health damage'),
 ('IMPACT World+ v2.0.1, footprint version',
  'water use',
  'water scarcity footprint'),
 ('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10',
  'Ecosystem quality',
  'Climate change, ecosystem quality, long term'),
 ('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10',
  'Ecosystem quality',
  'Climate change, ecosystem quality, short term'),
 ('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10',
  'Human health',
  'Climate change, human health, long term'),
 ('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10',
  'Human health',
  'Climate c

In [6]:
df_iw_methods = pd.DataFrame(IW_METHODS, columns=["Method", "Impact Category", "Subcategory"])
df_iw_methods.to_csv(r'data/iw2.1_methods.csv', index=False)

In [7]:
# Extract methods for IMPACT World+ Damage 2.1
IMPACT_METHODS_DAMAGE = [
    method for method in IW_METHODS 
    if method[0] == 'IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10'
]
IMPACT_METHODS_DAMAGE

[('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10',
  'Ecosystem quality',
  'Climate change, ecosystem quality, long term'),
 ('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10',
  'Ecosystem quality',
  'Climate change, ecosystem quality, short term'),
 ('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10',
  'Human health',
  'Climate change, human health, long term'),
 ('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10',
  'Human health',
  'Climate change, human health, short term'),
 ('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10',
  'Ecosystem quality',
  'Fisheries impact'),
 ('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10',
  'Ecosystem quality',
  'Freshwater acidification'),
 ('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10',
  'Ecosystem quality',
  'Freshwater ecotoxicity, long term'),
 ('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10',
  'Ecosystem quality',
  'Freshwater ecotoxicity, short

In [8]:
# Extract methods for IMPACT World+ Midpoint 2.1
IMPACT_METHODS_MIDPOINTS = [
    method for method in IW_METHODS 
    if method[0] == 'IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10'
]
IMPACT_METHODS_MIDPOINTS

[('IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10',
  'Midpoint',
  'Climate change, long term'),
 ('IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10',
  'Midpoint',
  'Climate change, short term'),
 ('IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10',
  'Midpoint',
  'Fossil and nuclear energy use'),
 ('IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10',
  'Midpoint',
  'Freshwater acidification'),
 ('IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10',
  'Midpoint',
  'Freshwater ecotoxicity'),
 ('IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10',
  'Midpoint',
  'Freshwater eutrophication'),
 ('IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10',
  'Midpoint',
  'Human toxicity cancer'),
 ('IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10',
  'Midpoint',
  'Human toxicity non-cancer'),
 ('IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10',
  'Midpoint',
  'Ionizing radiations'),
 ('IMPACT 

In [9]:
CC = [
    ('IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10', 'Midpoint', 'Climate change, long term'),
    ('IMPACT World+ Midpoint 2.1_regionalized for ecoinvent v3.10', 'Midpoint', 'Climate change, short term')
]

In [10]:
IW_EP = [
    ('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10', 'Human health', 'Total human health'),
    ('IMPACT World+ Damage 2.1_regionalized for ecoinvent v3.10', 'Ecosystem quality', 'Total ecosystem quality')
]

# LCA calculations for IEA energy transition scenarios

## Import metal quantities by scenarios and technology

In [11]:
#Import data from excel file
excel_file = r'data/data_metals.xlsx'
sheet_name = "metals_ei"
df_metals_ei = pd.read_excel(excel_file, sheet_name=sheet_name)

In [12]:
years = ["2022", "2025", "2030", "2035", "2040", "2045", "2050"]

In [13]:
#Multiply the quantities by 1e6 to have kilograms instead of kilotons
df_metals_ei[years] *= 1000000

In [14]:
#Demand by metal and by technology for the scenario NZE
df_NZE = df_metals_ei.loc[df_metals_ei['Scenario'] == 'NZE'].copy()
df_NZE

,Metal,Technology,Market,Reference Product,Location,Scenario,2022,2025,2030,2035,2040,2045,2050
0,Aluminium,Electricity Networks,"market for aluminium, wrought alloy (SM)","aluminium, wrought alloy",World,NZE,1.035700e+10,1.242300e+10,1.787200e+10,2.281800e+10,2.640300e+10,2.514800e+10,2.075900e+10
2,Arsenic,Solar PV,market for arsine,arsine,GLO,NZE,0.000000e+00,0.000000e+00,4.361360e+05,2.506000e+06,1.381130e+07,1.121710e+07,1.254430e+07
4,Boron,Wind,market for boric oxide,boric oxide,GLO,NZE,1.586180e+05,2.823520e+05,7.310900e+05,7.284700e+05,6.915950e+05,4.727970e+05,7.460790e+05
6,Cadmium,Solar PV,"market for cadmium, semiconductor-grade","cadmium, semiconductor-grade",GLO,NZE,4.208430e+05,7.703340e+05,1.224040e+06,1.132610e+06,9.542230e+05,5.988290e+05,4.311890e+05
8,Chromium,Wind,"market for ferrochromium, high-carbon, 68% Cr","ferrochromium, high-carbon, 68% Cr",GLO,NZE,5.031600e+07,9.078430e+07,1.947490e+08,1.843100e+08,1.503200e+08,1.045150e+08,1.550830e+08
10,Cobalt,Battery Storage,market for cobalt sulfate (SM),cobalt sulfate,World,NZE,3.588530e+06,6.229650e+06,1.702110e+07,1.772030e+07,1.443760e+07,6.950820e+06,0.000000e+00
11,Cobalt,EV,market for cobalt sulfate (SM),cobalt sulfate,World,NZE,6.456060e+07,1.304180e+08,1.879930e+08,2.308280e+08,2.439720e+08,2.782140e+08,2.905570e+08
12,Cobalt,Hydrogen,market for cobalt sulfate (SM),cobalt sulfate,World,NZE,1.073460e+04,1.511010e+05,3.020670e+05,2.059450e+05,8.079190e+04,5.541910e+04,7.586900e+04
16,Copper,Battery Storage,"market for copper, cathode (SM)","copper, cathode",World,NZE,2.044680e+07,7.749500e+07,2.581510e+08,4.408590e+08,6.853160e+08,7.338300e+08,6.651570e+08
17,Copper,Electricity Networks,"market for copper, cathode (SM)","copper, cathode",World,NZE,4.181610e+09,5.024260e+09,8.923600e+09,1.144450e+10,1.257440e+10,1.175690e+10,9.777840e+09


In [15]:
#Demand by metal and by technology for the scenario SPS
df_SPS = df_metals_ei.loc[df_metals_ei['Scenario'] == 'SPS'].copy()
df_SPS

,Metal,Technology,Market,Reference Product,Location,Scenario,2022,2025,2030,2035,2040,2045,2050
1,Aluminium,Electricity Networks,"market for aluminium, wrought alloy (SM)","aluminium, wrought alloy",World,SPS,1.035700e+10,1.160600e+10,1.452700e+10,1.448200e+10,1.417400e+10,1.494300e+10,1.434300e+10
3,Arsenic,Solar PV,market for arsine,arsine,GLO,SPS,0.000000e+00,0.000000e+00,1.903290e+05,1.084830e+06,6.381790e+06,7.385250e+06,8.349830e+06
5,Boron,Wind,market for boric oxide,boric oxide,GLO,SPS,1.586180e+05,1.757780e+05,3.149190e+05,2.877880e+05,2.597290e+05,3.295440e+05,4.082260e+05
7,Cadmium,Solar PV,"market for cadmium, semiconductor-grade","cadmium, semiconductor-grade",GLO,SPS,4.208430e+05,4.202920e+05,3.975620e+05,3.953310e+05,4.115000e+05,4.659760e+05,5.035290e+05
9,Chromium,Wind,"market for ferrochromium, high-carbon, 68% Cr","ferrochromium, high-carbon, 68% Cr",GLO,SPS,5.031600e+07,5.417720e+07,7.267500e+07,7.067570e+07,6.586560e+07,8.129220e+07,9.007150e+07
13,Cobalt,EV,market for cobalt sulfate (SM),cobalt sulfate,World,SPS,6.456060e+07,6.637010e+07,7.393740e+07,7.767280e+07,1.053090e+08,1.339870e+08,1.456130e+08
14,Cobalt,Battery Storage,market for cobalt sulfate (SM),cobalt sulfate,World,SPS,3.588530e+06,3.090400e+06,5.454650e+06,6.173410e+06,4.745430e+06,2.258910e+06,0.000000e+00
15,Cobalt,Hydrogen,market for cobalt sulfate (SM),cobalt sulfate,World,SPS,1.073460e+04,1.031570e+04,1.390280e+04,7.368120e+03,3.299830e+03,3.561640e+03,4.953660e+03
21,Copper,Solar PV,"market for copper, cathode (SM)","copper, cathode",World,SPS,6.816450e+08,7.789540e+08,9.072520e+08,9.248000e+08,9.587660e+08,1.122060e+09,1.261660e+09
22,Copper,Wind,"market for copper, cathode (SM)","copper, cathode",World,SPS,3.937920e+08,4.277650e+08,6.463330e+08,5.952550e+08,5.322900e+08,6.390060e+08,7.206340e+08


In [14]:
#df_NZE.to_csv(r'data/df_NZE.csv', index=False)
#df_SPS.to_csv(r'data/df_SPS.csv', index=False)

In [16]:
# Total demand by metal, for all technologies for the NZE scenario
df_NZE_all = df_NZE.groupby('Metal')[years].sum()

In [17]:
# Total demand by metal, for all technologies for the SPS scenario
df_SPS_all = df_SPS.groupby('Metal')[years].sum()

## LCA calculations

Create a dictionary with all the EI activities (for each metals) and their keys

In [21]:
def setup_activities_dict(df, scenario_prefix):
    """
    Create a dictionary mapping (scenario, year, metal, technology) to Brightway activities.
    """
    activities_dict = {}
    
    for index, row in df.iterrows():
        metal = row["Metal"]
        market = row["Market"]
        product = row["Reference Product"]
        location = row["Location"]
        
        for year in years:
            database_name = f"{scenario_prefix}_{year} regionalized"  # Select the appropriate database
            activity_name = (scenario_prefix, year, metal, row["Technology"])
            search_criteria = market
            
            try:
                activities = Database(database_name).search(search_criteria, limit=1000)
                filtered_activities = [
                    i for i in activities if i["name"] == search_criteria
                    and i["location"] == location
                    and i["reference product"] == product
                ]
                
                if filtered_activities:
                    activities_dict[activity_name] = filtered_activities[0]
                else:
                    activities_dict[activity_name] = None
            except:
                activities_dict[activity_name] = None
    
    return activities_dict

In [22]:
sps_activities_dict = setup_activities_dict(df_SPS, scenario_prefix='SPS')
nze_activities_dict = setup_activities_dict(df_NZE, scenario_prefix='NZE')

In [24]:
def calculate_lca(df, activities_dict, scenario_prefix, lcia_methods):
    """
    Perform LCA calculations for different scenarios, years, metals, and technologies.
    Returns a DataFrame with results disaggregated.
    """
    results = []
    
    for index, row in df.iterrows():
        metal = row["Metal"]
        technology = row["Technology"]
        
        for year in years:
            quantity = row[str(year)]  # Get the metal quantity for the year
            activity_name = (scenario_prefix, year, metal, technology)
            activity = activities_dict.get(activity_name, None)
            
            if activity:
                impact_results = {}
                
                for method in lcia_methods:
                    try:
                        lca = LCA({activity: quantity}, method)
                        lca.lci()
                        lca.lcia()
                        impact_results[method[2]] = lca.score
                    except:
                        impact_results[method[2]] = np.nan  # Assign NaN if calculation fails
                
                results.append({
                    "Scenario": scenario_prefix,
                    "Year": year,
                    "Metal": metal,
                    "Technology": technology,
                    **impact_results
                })
    
    return pd.DataFrame(results)

In [25]:
def calculate_lca_optimized(df, activities_dict, scenario_prefix, lcia_methods):
    """
    Optimized LCA calculations for different scenarios, years, metals, and technologies.
    """
    results = []
    
    for index, row in df.iterrows():
        metal = row["Metal"]
        technology = row["Technology"]
        
        for year in years:
            quantity = row[str(year)]  # Get the metal quantity for the year
            activity_name = (scenario_prefix, year, metal, technology)
            activity = activities_dict.get(activity_name, None)
            
            if activity:
                impact_results = {}

                try:
                    # Initialize LCA object once per activity
                    lca = LCA({activity: quantity}, lcia_methods[0])  # Use first method as reference
                    lca.lci()

                    for method in lcia_methods:
                        lca.switch_method(method)  # Switch to a new impact method without recalculating LCI
                        lca.lcia()
                        impact_results[method[2]] = lca.score
                
                except:
                    for method in lcia_methods:
                        impact_results[method[2]] = np.nan  # Assign NaN if calculation fails

                results.append({
                    "Scenario": scenario_prefix,
                    "Year": year,
                    "Metal": metal,
                    "Technology": technology,
                    **impact_results
                })
    
    return pd.DataFrame(results)


In [26]:
df_sps_damage = calculate_lca_optimized(df_SPS, sps_activities_dict, scenario_prefix='SPS', lcia_methods=IMPACT_METHODS_DAMAGE)

In [27]:
df_sps_damage

,Scenario,Year,Metal,Technology,"Climate change, ecosystem quality, long term","Climate change, ecosystem quality, short term","Climate change, human health, long term","Climate change, human health, short term",Fisheries impact,Freshwater acidification,...,"Photochemical ozone formation, human health",Terrestrial acidification,"Terrestrial ecotoxicity, long term","Terrestrial ecotoxicity, short term",Thermally polluted water,"Water availability, freshwater ecosystem","Water availability, human health","Water availability, terrestrial ecosystem",Total human health,Total ecosystem quality
0,SPS,2022,Aluminium,Electricity Networks,4.561995e+10,1.357305e+10,614818.830824,182923.616561,0.004428,1.038317e+09,...,180.500840,8.157259e+09,2.370977e+08,2.007880e+08,3.023031e+07,177602.271175,3850.489610,4.394529e+06,980519.946440,1.080706e+11
1,SPS,2025,Aluminium,Electricity Networks,4.370999e+10,1.294177e+10,589078.310624,174415.869932,0.006092,1.080773e+09,...,176.821888,8.237943e+09,2.573087e+08,2.189858e+08,3.464903e+07,180989.273841,4151.326119,4.054055e+06,941758.449719,1.048242e+11
2,SPS,2030,Aluminium,Electricity Networks,4.609567e+10,1.357444e+10,621230.032998,182942.322265,0.008514,1.245750e+09,...,192.088707,9.282564e+09,3.107067e+08,2.688506e+08,3.721695e+07,198510.721031,4879.589942,4.090366e+06,997309.699880,1.125863e+11
3,SPS,2035,Aluminium,Electricity Networks,3.993314e+10,1.169953e+10,538177.778132,157674.219039,0.008874,1.164536e+09,...,172.142563,8.537230e+09,3.020693e+08,2.647670e+08,3.283212e+07,179070.005523,4696.511486,3.382868e+06,869338.936731,9.924247e+10
4,SPS,2040,Aluminium,Electricity Networks,3.373272e+10,9.821564e+09,454614.945773,132364.949771,0.009050,1.082091e+09,...,151.926354,7.762275e+09,2.894522e+08,2.565913e+08,3.009594e+07,159526.993363,4410.079036,2.712354e+06,741482.911595,8.562836e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,SPS,2030,Zirconium,Hydrogen,7.682338e+05,2.320058e+05,10.353465,3.126735,0.000002,2.305904e+04,...,0.005345,1.294479e+05,1.303395e+04,2.164601e+04,5.724057e+02,24.993770,1.557567,2.229144e+03,19.628006,3.193359e+06
346,SPS,2035,Zirconium,Hydrogen,5.315203e+05,1.605532e+05,7.163285,2.163772,0.000001,1.708841e+04,...,0.003975,9.337944e+04,1.031459e+04,1.739504e+04,4.333488e+02,19.309960,1.240771,1.771698e+03,13.995715,2.379546e+06
347,SPS,2040,Zirconium,Hydrogen,4.112869e+05,1.240814e+05,5.542903,1.672241,0.000001,1.455436e+04,...,0.003365,7.720753e+04,9.214868e+03,1.576056e+04,3.831568e+02,16.923087,1.113369,1.585694e+03,11.242318,2.003706e+06
348,SPS,2045,Zirconium,Hydrogen,3.818587e+05,1.148750e+05,5.146300,1.548167,0.000001,1.501273e+04,...,0.003457,7.675956e+04,9.889844e+03,1.716437e+04,4.122648e+02,17.917763,1.201580,1.709151e+03,10.874702,2.036938e+06


In [28]:
df_sps_midpoints = calculate_lca_optimized(df_SPS, sps_activities_dict, scenario_prefix='SPS', lcia_methods=IMPACT_METHODS_MIDPOINTS)

In [29]:
df_sps_midpoints

,Scenario,Year,Metal,Technology,"Climate change, long term","Climate change, short term",Fossil and nuclear energy use,Freshwater acidification,Freshwater ecotoxicity,Freshwater eutrophication,...,Ionizing radiations,"Land occupation, biodiversity","Land transformation, biodiversity",Marine eutrophication,Mineral resources use,Ozone layer depletion,Particulate matter formation,Photochemical ozone formation,Terrestrial acidification,Water scarcity
0,SPS,2022,Aluminium,Electricity Networks,1.107812e+11,1.186879e+11,1.126542e+12,3.727916e+08,3.275332e+13,701373.309649,...,3.725555e+11,1.292869e+09,6.334728e+06,7.747833e+06,6.037950e+08,876.480440,1.034656e+08,1.983526e+08,6.193024e+08,1.328265e+10
1,SPS,2025,Aluminium,Electricity Networks,1.060498e+11,1.132015e+11,1.095214e+12,3.898258e+08,3.238743e+13,700370.431057,...,4.359988e+11,1.348095e+09,7.281226e+06,7.560159e+06,6.482658e+08,921.209987,9.792598e+07,1.943098e+08,6.219208e+08,1.381816e+10
2,SPS,2030,Aluminium,Electricity Networks,1.117276e+11,1.188163e+11,1.158454e+12,4.531021e+08,3.593218e+13,798322.179381,...,5.225124e+11,1.604623e+09,9.554732e+06,8.216924e+06,7.801004e+08,1059.764478,1.023001e+08,2.110865e+08,6.966283e+08,1.558064e+10
3,SPS,2035,Aluminium,Electricity Networks,9.670156e+10,1.024765e+11,1.004405e+12,4.254425e+08,3.256987e+13,747078.764840,...,5.019137e+11,1.559732e+09,9.898360e+06,7.356988e+06,7.593087e+08,988.015251,8.929291e+07,1.891677e+08,6.387024e+08,1.445071e+10
4,SPS,2040,Aluminium,Electricity Networks,8.159636e+10,8.608936e+10,8.487728e+11,3.990486e+08,2.895157e+13,686358.616894,...,4.783271e+11,1.500493e+09,9.974422e+06,6.478276e+06,7.228783e+08,914.302959,7.734625e+07,1.669520e+08,5.796131e+08,1.308186e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,SPS,2030,Zirconium,Hydrogen,1.873715e+06,2.007657e+06,2.599659e+07,8.891439e+03,5.754033e+08,147.660549,...,1.964031e+07,5.185540e+05,6.087039e+03,2.979087e+02,8.809398e+05,0.193706,2.583516e+03,5.873839e+03,9.301120e+03,3.887305e+06
346,SPS,2035,Zirconium,Hydrogen,1.296720e+06,1.389769e+06,1.823057e+07,6.656371e+03,4.232053e+08,117.881293,...,1.396699e+07,4.178935e+05,4.867470e+03,2.280343e+02,7.037224e+05,0.153663,1.902155e+03,4.368211e+03,6.718426e+03,3.066093e+06
347,SPS,2040,Zirconium,Hydrogen,1.003442e+06,1.073842e+06,1.428867e+07,5.751632e+03,3.514296e+08,106.125512,...,1.127902e+07,3.806994e+05,4.378647e+03,1.978923e+02,6.323800e+05,0.138596,1.596043e+03,3.697946e+03,5.551142e+03,2.729331e+06
348,SPS,2045,Zirconium,Hydrogen,9.314636e+05,9.934169e+05,1.344348e+07,6.054039e+03,3.497659e+08,114.871439,...,1.102349e+07,4.172779e+05,4.734812e+03,2.075571e+02,6.836267e+05,0.148594,1.613303e+03,3.798779e+03,5.510405e+03,2.926264e+06


In [30]:
df_nze_damage = calculate_lca_optimized(df_NZE, nze_activities_dict, scenario_prefix='NZE', lcia_methods=IMPACT_METHODS_DAMAGE)

In [33]:
df_nze_midpoints = calculate_lca_optimized(df_NZE, nze_activities_dict, scenario_prefix='NZE', lcia_methods=IMPACT_METHODS_MIDPOINTS)

In [34]:
# Save to csv to the result folder
df_sps_damage.to_csv(r'results/df_sps_damage_regionalized.csv', index=False)
df_sps_midpoints.to_csv(r'results/df_sps_midpoints_regionalized.csv', index=False)
df_nze_damage.to_csv(r'results/df_nze_damage_regionalized.csv', index=False)
df_nze_midpoints.to_csv(r'results/df_nze_midpoints_regionalized.csv', index=False)

# LCA calculation for the all economy